In [0]:
import importlib
import src.load.aggregate_gold as gold_module
importlib.reload(gold_module)
transform_run = gold_module.run
#for file reload issue

In [0]:
import yaml
from src.load.aggregate_gold import run as transform_run

# Loading cofiguration file
with open("/Workspace/Repos/ziaarzoo21@gmail.com/spark-etl/config/config.yaml") as f:
    cfg = yaml.safe_load(f)



In [0]:
gold_table = transform_run(cfg, spark)
df_gold = spark.table(gold_table)
display(df_gold.limit(10))

spark.sql(f"DESCRIBE DETAIL {gold_table}").display()